In [1]:
import numpy as np
import pufferlib, pufferlib.vector
from pufferlib.environments import classic_control

In [2]:
num_envs = 12

In [3]:
vecenv = pufferlib.vector.make(
    classic_control.env_creator("CartPole-v1"),
    num_envs=num_envs,
    backend=pufferlib.vector.Multiprocessing,
)

In [4]:
initial_obs, _ = vecenv.reset()
initial_obs.shape, initial_obs

((12, 4),
 array([[ 0.0273956 , -0.00611216,  0.03585979,  0.0197368 ],
        [ 0.01522993, -0.04562247, -0.04799704,  0.03392126],
        [-0.03774345, -0.02418869, -0.00942293,  0.0469184 ],
        [ 0.00731307,  0.00284911,  0.02636502,  0.03116928],
        [ 0.04056043, -0.04227732, -0.02274302,  0.01218505],
        [ 0.0241802 ,  0.02536688, -0.00348195, -0.0396275 ],
        [-0.01123001,  0.00957735,  0.00138005,  0.01944919],
        [-0.01371458,  0.00932173, -0.01080498,  0.01236993],
        [ 0.02874227,  0.03336693,  0.00479045,  0.04734491],
        [ 0.04424412, -0.01928616, -0.02450934, -0.02773815],
        [ 0.0115763 ,  0.00742263,  0.00195436, -0.04936105],
        [-0.0488873 ,  0.02189106, -0.01688728,  0.04330887]],
       dtype=float32))

In [5]:
n_input = vecenv.single_observation_space.shape[0]
n_actions = vecenv.single_action_space.n

In [6]:
n_input, n_actions

(4, 2)

In [7]:
env_rewards = [list() for _ in range(num_envs)]
env_rewards

returns = []

from itertools import count

dones = np.array([False] * num_envs)
truncateds = np.array([False] * num_envs)
dones, truncateds

obs, _ = vecenv.reset()
episodes = 0
for t in count():
    obs, rewards, dones, truncateds, _ = vecenv.step(vecenv.action_space.sample())
    for i, reward in enumerate(rewards):
        env_rewards[i].append(reward)
    for i in np.where(dones | truncateds)[0]:
        returns.append(sum(env_rewards[i]))
        env_rewards[i] = []
        episodes += 1
    if episodes >= 1_000:
        break

np.mean([np.sum(rets) for rets in returns]), t

(22.164, 1950)

In [8]:
import torch
from torch import nn

In [9]:
class QNetwork(nn.Module):
    def __init__(self, n_input, n_hiddens, n_actions):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(n_input, n_hiddens),
            nn.ReLU(),
            nn.Linear(n_hiddens, n_actions),
        )

    def forward(self, x):
        device = next(self.mlp.parameters()).device
        x = torch.tensor(np.array(x), dtype=torch.float32, device=device)
        
        return self.mlp(x)

In [10]:
n_hiddens = 8

In [11]:
net = QNetwork(n_input, n_hiddens, n_actions)
target_net = QNetwork(n_input, n_hiddens, n_actions)
target_net.load_state_dict(net.state_dict())
target_net.eval()
net, target_net

(QNetwork(
   (mlp): Sequential(
     (0): Linear(in_features=4, out_features=8, bias=True)
     (1): ReLU()
     (2): Linear(in_features=8, out_features=2, bias=True)
   )
 ),
 QNetwork(
   (mlp): Sequential(
     (0): Linear(in_features=4, out_features=8, bias=True)
     (1): ReLU()
     (2): Linear(in_features=8, out_features=2, bias=True)
   )
 ))

In [12]:
obs, _ = vecenv.reset()
with torch.no_grad():
    print(torch.all(net(obs) == target_net(obs)))

tensor(True)


In [13]:
import random

In [14]:
def select_actions(q_values, epsilon):
    batch_size, n_actions = q_values.shape
    if random.random() < epsilon:
        return np.random.choice(range(n_actions), batch_size)
    return np.array(q_values.argmax(dim=-1).cpu())

In [15]:
with torch.no_grad():
    q_values = net(obs)
    print(select_actions(q_values, 0.5))

[0 0 0 0 0 0 0 0 0 0 0 0]


In [16]:
from collections import deque

In [17]:
class ReplayBuffer:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.queue = deque(maxlen=capacity)

    def store(self, transition):
        self.queue.append(transition)

    def extend(self, transitions):
        for transition in transitions:
            self.store(transition)

    def sample(self, n_samples: int):
        return [random.choice(self.queue) for _ in range(n_samples)]

    def __len__(self):
        return len(self.queue)

    def __repr__(self):
        return self.queue.__repr__()

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch import optim

from itertools import count
from tqdm.auto import tqdm

net = QNetwork(n_input, n_hiddens, n_actions).to(device)
target_net = QNetwork(n_input, n_hiddens, n_actions).to(device)
target_net.load_state_dict(net.state_dict())
target_net.eval()


env_rewards = [list() for _ in range(num_envs)]
env_rewards
returns = []

gamma = 1.0
n_target_update_eps = 60
n_episodes = 4_800
buffer_size = 100_000
update_batch_size = 512
learning_rate = 1e-4
epsilon_start = 1.0
epsilon_end = 0.0
epsilon_decay = 9 * n_episodes // 10
epsilon_decay_rate = (epsilon_start - epsilon_end) / epsilon_decay
epsilon = epsilon_start

optimizer = optim.Adam(net.parameters(), lr=learning_rate)
loss_fn = nn.SmoothL1Loss()
buffer = ReplayBuffer(capacity=buffer_size)

obs, _ = vecenv.reset()
obs = obs.copy()
episodes = 0
bar = tqdm(
    desc="Episodes",
    total=n_episodes,
    initial=0,
)

debug = False
debug_seen = False

for t in count():
    with torch.no_grad():
        q_values = net(obs)
        actions = select_actions(q_values, epsilon)

    if debug and debug_seen:
        print("Ob seen:\t", obs[0])
        print("Action taken:\t", actions[0])
    
    next_obs, rewards, dones, truncateds, _ = vecenv.step(actions)

    if debug and debug_seen:
        print("Ob seen:\t", obs[0])
    buffer.extend(zip(obs.copy(), actions.copy(), rewards.copy(), next_obs.copy(), (dones | truncateds).copy()))
    obs = next_obs.copy()

    for i, reward in enumerate(rewards):
        env_rewards[i].append(reward)

    if len(buffer) >= update_batch_size:
        obs_s, actions_s, rewards_s, next_obs_s, dones_s = zip(*buffer.sample(update_batch_size))
        if debug:
            print("\nOb:\t\t", obs_s[0])
            print("Action:\t", actions_s[0])
            print("Reward:\t", rewards_s[0])
            print("Next ob:\t", next_obs_s[0])
            print("Done:\t\t", dones_s[0])
    
            print("\nBuffer length:", len(buffer))
            print("\nBuffer items:", buffer.queue[0])

        actions_t = torch.tensor(actions_s, device=device).unsqueeze(1)
        q_values_t = net(obs_s).gather(1, actions_t).squeeze(1)
        rewards_t = torch.tensor(rewards_s, device=device) / 500.
        dones_t = torch.tensor(dones_s, device=device)
        with torch.no_grad():
            target_q_values_t = target_net(next_obs_s).max(dim=1).values

        target = rewards_t + gamma * ~dones_t * target_q_values_t
        loss = loss_fn(target, q_values_t)

        if debug:
            print("Actions T:\t", actions_t[0])
            print("Qs T:\t\t", q_values_t[0])
            print("Target Qs:\t", target_q_values_t[0])
            print("TD Target:\t", target[0])
            with torch.no_grad():
                td_error = target - q_values_t
            print("TD Error:\t", td_error[0])


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if debug:
            with torch.no_grad():
                q_values_t = net(obs_s).gather(1, actions_t).squeeze(1)
                target_q_values_t = target_net(next_obs_s).max(dim=1).values
                target = rewards_t + gamma * ~dones_t * target_q_values_t
                td_error = target - q_values_t
                
            print("Actions T:\t", actions_t[0])
            print("Qs T:\t\t", q_values_t[0])
            print("TD Target:\t", target[0])
            print("Target Qs:\t", target_q_values_t[0])
            print("TD Error:\t", td_error[0])
    
            break
        
    for i in np.where(dones | truncateds)[0]:
        returns.append(sum(env_rewards[i]))
        env_rewards[i] = []
        episodes += 1
        if epsilon >= epsilon_end:
            epsilon -= epsilon_decay_rate
        bar.update()
        
    if episodes >= n_episodes:
        bar.close()
        debug = True
        #break

    if episodes % n_target_update_eps == 0:
        target_net.load_state_dict(net.state_dict())

np.max([np.sum(rets) for rets in returns]), t

Episodes:   0%|          | 0/4800 [00:00<?, ?it/s]

In [ ]:
np.mean([np.sum(rets) for rets in returns[-100:]])

In [ ]:
import gymnasium as gym
import numpy as np
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
video_folder = f"./videos/DQN_scratch_CartPole_{timestamp}"

env = gym.make("CartPole-v1", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, video_folder)

epsilon = epsilon_end

for _ in range(9):
    ob, _ = env.reset()
    ob = np.expand_dims(ob, 0)
    ret = 0
    done, truncated = False, False
    while not (done or truncated):
        with torch.no_grad():
            q_values = net(ob)
        actions = select_actions(q_values, epsilon)
        ob, reward, done, truncated, _ = env.step(actions[0])
        ob = np.expand_dims(ob, 0)
        ret += reward
    
    print(ret)

env.close()